# March Madness Predictor Kaggle Competition
This script shows the set-up, preprocessing and data storage for model training.
#### Dependencies

In [2]:
# KenPom Scraper
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

# sklearn 
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
# tensorflow (r0.12)
import tensorflow as tf

## Scrape KenPom website for Team Statistics 
Retrieved during the 2016-2017 competition # find source!

In [3]:
try :
    KP = pd.read_csv('kenpom.csv')
    print("KenPom scrape was completed previously and retrieved from local csv file")
except :

    # Base url, and a lambda func to return url for a given year
    base_url = 'http://kenpom.com/index.php'
    url_year = lambda x: '%s?y=%s' % (base_url, str(x) if x != 2016 else base_url)

    # Years on kenpom's site (could also scrape this and 
    # set as a list if you want to be more dynamic)
    years = range(2002, 2018)

    #%%# Create a method that parses a given year and spits out a raw dataframe
    def import_raw_year(year):
        """
        Imports raw data from a ken pom year into a dataframe
        """
        f = requests.get(url_year(year))
        soup = BeautifulSoup(f.text, "lxml")
        table_html = soup.find_all('table', {'id': 'ratings-table'})

        # Weird issue w/ <thead> in the html
        # Prevents us from just using pd.read_html
        # Let's find all the thead contents and just replace/remove them
        # This allows us to easily put the table row data into a dataframe using panda
        thead = table_html[0].find_all('thead')

        table = table_html[0]
        for x in thead:
            table = str(table).replace(str(x), '')

    #    table = "<table id='ratings-table'>%s</table>" % table
        df = pd.read_html(table)[0]
        df['year'] = year
        return df
    #%%
        # Import all the years into a singular dataframe
    df = None
    for x in years:
        df = pd.concat( (df, import_raw_year(x)), axis=0) \
            if df is not None else import_raw_year(2002)
    #%%
    df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
                   'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
                   'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
                   'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
                   'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']
    #%%
    # Lambda that returns true if given string is a number and a valid seed number (1-16)
    valid_seed = lambda x: True if str(x).replace(' ', '').isdigit() \
                    and int(x) > 0 and int(x) <= 16 else False

    # Use lambda to parse out seed/team
    df['Seed'] = df['Team'].apply(lambda x: x[-2:].replace(' ', '') \
                                  if valid_seed(x[-2:]) else np.nan )

    df['Team'] = df['Team'].apply(lambda x: x[:-2] if valid_seed(x[-2:]) else x)
    #%%       
    # Split W-L column into wins and losses
    df['Wins'] = df['W-L'].apply(lambda x: int(re.sub('-.*', '', x)) )
    df['Losses'] = df['W-L'].apply(lambda x: int(re.sub('.*-', '', x)) )
    df.drop('W-L', inplace=True, axis=1)
    #%%     
    # Reorder columns just cause I'm OCD
    df=df[[ 'Year', 'Rank', 'Team', 'Conference', 'Wins', 'Losses', 'Seed','Pyth', 
                 'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
                 'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
                 'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
                 'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank']]
    #%%
    df.to_csv('kenpom.csv', index=False)
    #%%     
    KP = df.copy()
    del df
    
    print("KenPom Database downloaded and stored as 'kenpom.csv' in local directory.")

        


KenPom scrape was completed previously and retrieved from local csv file


## Scale KenPom data
Scale to min and max of [0,1]

In [4]:
categories = KP.columns
print(categories)

Index(['Year', 'Rank', 'Team', 'Conference', 'Wins', 'Losses', 'Seed', 'Pyth',
       'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank', 'AdjustT',
       'AdjustT Rank', 'Luck', 'Luck Rank', 'SOS Pyth', 'SOS Pyth Rank',
       'SOS OppO', 'SOS OppO Rank', 'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth',
       'NCSOS Pyth Rank'],
      dtype='object')


In [5]:
KP.head()

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
0,2002,1,Duke,ACC,31,4,1.0,34.14,118.9,1,...,-0.027,223,10.09,20,107.8,15,97.7,39,7.52,18
1,2002,2,Cincinnati,CUSA,31,4,1.0,30.15,116.0,7,...,0.002,165,7.45,54,105.2,60,97.8,42,5.41,43
2,2002,3,Maryland,ACC,32,4,1.0,29.24,117.2,3,...,0.025,104,10.32,18,107.9,11,97.6,35,5.12,45
3,2002,4,Kansas,B12,31,4,1.0,28.95,116.7,5,...,0.023,109,10.75,13,108.8,7,98.1,49,8.91,9
4,2002,5,Oklahoma,B12,31,5,2.0,26.02,113.0,20,...,0.044,67,9.39,29,107.8,14,98.4,61,2.55,87


In [6]:
drop_bool = [x.find('Rank') != -1 for x in categories]
categories[drop_bool]

Index(['Rank', 'AdjustO Rank', 'AdjustD Rank', 'AdjustT Rank', 'Luck Rank',
       'SOS Pyth Rank', 'SOS OppO Rank', 'SOS OppD Rank', 'NCSOS Pyth Rank'],
      dtype='object')

In [7]:
columns_to_drop = categories[drop_bool].tolist()
columns_to_drop += ['Conference', 'Seed'] 
columns_to_drop

['Rank',
 'AdjustO Rank',
 'AdjustD Rank',
 'AdjustT Rank',
 'Luck Rank',
 'SOS Pyth Rank',
 'SOS OppO Rank',
 'SOS OppD Rank',
 'NCSOS Pyth Rank',
 'Conference',
 'Seed']

In [8]:
KP.drop(columns_to_drop, axis = 1, inplace = True)
print(KP.head())

   Year        Team  Wins  Losses   Pyth  AdjustO  AdjustD  AdjustT   Luck  \
0  2002        Duke    31       4  34.14    118.9     84.7     75.9 -0.027   
1  2002  Cincinnati    31       4  30.15    116.0     85.9     68.6  0.002   
2  2002    Maryland    32       4  29.24    117.2     88.0     74.7  0.025   
3  2002      Kansas    31       4  28.95    116.7     87.8     78.4  0.023   
4  2002    Oklahoma    31       5  26.02    113.0     87.0     68.2  0.044   

   SOS Pyth  SOS OppO  SOS OppD  NCSOS Pyth  
0     10.09     107.8      97.7        7.52  
1      7.45     105.2      97.8        5.41  
2     10.32     107.9      97.6        5.12  
3     10.75     108.8      98.1        8.91  
4      9.39     107.8      98.4        2.55  


In [9]:
X_keys = KP[['Year', 'Team']]
X_data = KP.drop(['Year', 'Team'], axis = 1)

In [10]:
"""Scale X_data mean zero, scale variation based on IQR"""
rob_scaler = RobustScaler()
X_data_scaled = rob_scaler.fit_transform(X_data)


In [11]:
print(X_data.columns)
print(X_data_scaled[0:4])

Index(['Wins', 'Losses', 'Pyth', 'AdjustO', 'AdjustD', 'AdjustT', 'Luck',
       'SOS Pyth', 'SOS OppO', 'SOS OppD', 'NCSOS Pyth'],
      dtype='object')
[[ 1.5        -1.57142857  2.09897405  1.79411765 -1.75        1.8372093
  -0.41176471  1.15854922  1.24074074 -0.78        1.36395147]
 [ 1.5        -1.57142857  1.85817743  1.50980392 -1.61956522  0.13953488
   0.01470588  0.88497409  0.75925926 -0.76        0.9982669 ]
 [ 1.6        -1.57142857  1.8032589   1.62745098 -1.39130435  1.55813953
   0.35294118  1.18238342  1.25925926 -0.8         0.94800693]
 [ 1.5        -1.57142857  1.78575739  1.57843137 -1.41304348  2.41860465
   0.32352941  1.22694301  1.42592593 -0.7         1.60485269]]


## Generate dictionary for Team to TeamID conversion

In [12]:
team_ids = pd.read_csv('teams.csv')

In [13]:
id2team_list = list(zip(team_ids['Team_Id'], team_ids['Team_Name']))
team2id_list = list(zip(team_ids['Team_Name'], team_ids['Team_Id']))

In [14]:
team2id_dict = dict(team2id_list)
id2team_dict = dict(id2team_list)

#### Test every Team name key in the stats_dict is in the team_dict
The loop structure below asks the user to search and update team keys as needed for proper alignment between the stats dictionary and the team names provided by Kaggle data documents.

In [21]:
regex_dict = {}

for ix in X_keys.index :
    team = X_keys['Team'].iloc[ix]
    if team2id_dict.get(team) is not None :
        pass
    else: # look for previous regex
        print("="*80)
        print(team)
        try :
            new_key = regex_dict.get(team)
        except :
            new_key = team
        if team2id_dict.get(new_key) is not None :
            regex_dict.update({team : new_key})
        else : # strip spaces and punctuation
            new_key = team.strip().rstrip('.')
            if team2id_dict.get(new_key) is not None :
                regex_dict.update({team : new_key})
            else : # ask for input
                go = True
                while go :
                    query = input("\nSearch Query:\n")
                    print("")
                    for x in team2id_dict :
                        if x.find(query) != -1 :
                            print(x)
                    stop = input("Found it? [y/n]\n")
                    if stop == 'y' :
                        go = False
                new_key = input("Key?\n")
                assert team2id_dict.get(new_key) is not None, 'ERROR, you screwed up somewhere.'
                regex_dict.update({team : new_key}) 
        X_keys.ix[ix,'Team'] = new_key
                        
        
        

Kent St. 

Search Query:
Kent

Kentucky
E Kentucky
N Kentucky
Kent
Found it? [y/n]
y
Key?
Kent


/Users/ccthomps/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Ohio St.
North Carolina St.

Search Query:
NC

NC State
UNC Wilmington
NC A&T
UNC Asheville
UNC Greensboro
NC Central
Found it? [y/n]
y
Key?
NC State
Mississippi St.
Missouri 
Michigan St. 
Oklahoma St.
Tulsa 
Western Kentucky

Search Query:
WKU

WKU
Found it? [y/n]
y
Key?
WKU
Utah 
Valparaiso 
Southern Illinois 

Search Query:
S Illi

S Illinois
Found it? [y/n]
y
Key?
S Illinois
Hawaii 
Ball St.
Penn 
Creighton 
Pepperdine 
UNC Wilmington 
Wyoming 
Saint Joseph's

Search Query:
Joseph

St Joseph's PA
Found it? [y/n]
y
Key?
St Joseph's PA
Arizona St.
Boston College 
San Diego St. 
Utah St.
St. John's

Search Query:
John

St John's
Found it? [y/n]
y
Key?
St John's
UC Santa Barbara 

Search Query:
Santa

Santa Barbara
Santa Clara
Found it? [y/n]
y
Key?
Santa Barbara
St. Bonaventure

Search Query:
Bona

St Bonaventure
Found it? [y/n]
y
Key?
St Bonaventure
College of Charleston

Search Query:
Charl

Charlotte
Col Charleston
Charleston So
Found it? [y/n]
y
Key?
Col Charleston
Fresno St.
Sai

In [23]:
X_keys.tail()

,Year,Team
5448,2017,Longwood
5449,2017,Ark Pine Bluff
5450,2017,NC A&T
5451,2017,Presbyterian
5452,2017,Alabama A&M


In [24]:
X_keys.to_csv('Team_names.csv')

## Generate dictionary for stats retrieval 

In [25]:
stats_dict = {}
for year in X_keys['Year'].unique() :
    year_dict = {}
    for ix in X_keys.ix[X_keys['Year']==year, :].index :
        stats_list = list(zip(X_data.columns, X_data_scaled[ix,:]))
        team_name = X_keys['Team'].iloc[ix].strip().rstrip('.')
        year_dict.update({team_name : dict(stats_list)})
    
    stats_dict.update({year: year_dict})
print("Test for dictionary generation: {}".format(len(stats_dict) == len(X_keys['Year'].unique())))
    

Test for dictionary generation: True


In [26]:
for year in stats_dict :
    print("{} : {} teams".format(year, len(stats_dict[year])))

2016 : 350 teams
2017 : 350 teams
2002 : 326 teams
2003 : 326 teams
2004 : 325 teams
2005 : 329 teams
2006 : 333 teams
2007 : 335 teams
2008 : 340 teams
2009 : 343 teams
2010 : 346 teams
2011 : 344 teams
2012 : 344 teams
2013 : 346 teams
2014 : 350 teams
2015 : 350 teams


In [27]:
import pickle
with open('stats_dictionary.pickle', 'wb') as f :
    pickle.dump(stats_dict, f)


In [28]:
with open('team2id.pickle', 'wb') as f :
    pickle.dump(team2id_dict, f)
with open('id2team.pickle', 'wb') as f :
    pickle.dump(id2team_dict, f)

## Notes
This notebook contains the steps to download, process and store yearly statistical data from expert Ken Pomeroy's database.  Statistics were stored in a pickled dictionary that can be accessed using team name keys that are consistent with the names provided from the competition data and stored in the team2id and id2team dictionaries.

The next step will be to use these sources for neural network training, then prediction.